In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns

import itertools


#For random forest for time series:
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

#For metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

#Having to import from subfolders of parent directories is a nightmare
import sys
import os
current = os.path.dirname(os.path.realpath("Single-House-Optimization.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")

#From Functions folder
from LoadSeries import load_series, moving_average
from Merge import merge
from Battery import Battery
from DPModel import DPModel
from Logic import logic_bat, logic_DP
from Funcs_Logic_DP import (logic_rollout, get_price, print_price_summary, 
                           logic_series_print, policy_rollout, DP_stochastic, pred_logic_rollout)

In [2]:
house = "h16"
prod = load_series("p",house,"prod_"+house,[24,48,168])
cons = load_series("c",house,"cons_"+house,[24,48,168])
cons

,cons_h16,cons_h16-24,cons_h16-48,cons_h16-168,Year,Month,Day,Weekday,Hour
2020-12-29 00:00:00,5.7,5.7,6.9,6.3,2020,12,29,1,0
2020-12-29 01:00:00,5.9,5.4,5.3,4.8,2020,12,29,1,1
2020-12-29 02:00:00,5.0,6.0,4.9,4.8,2020,12,29,1,2
2020-12-29 03:00:00,3.9,6.8,4.4,4.3,2020,12,29,1,3
2020-12-29 04:00:00,3.5,4.0,3.6,4.0,2020,12,29,1,4
...,...,...,...,...,...,...,...,...,...
2023-01-10 16:00:00,6.1,7.6,6.9,5.4,2023,1,10,1,16
2023-01-10 17:00:00,8.3,10.0,9.8,6.2,2023,1,10,1,17
2023-01-10 18:00:00,10.4,9.1,14.5,13.0,2023,1,10,1,18
2023-01-10 19:00:00,9.9,11.1,10.8,10.4,2023,1,10,1,19


In [12]:
merged = merge(house)
merged

,prod_h16,cons_h16,power_yield,SpotPriceDKK
2020-12-22 00:00:00,0.0,6.3,-6.3,11.090000
2020-12-22 01:00:00,0.0,4.8,-4.8,-0.150000
2020-12-22 02:00:00,0.0,4.8,-4.8,-55.880001
2020-12-22 03:00:00,0.0,4.3,-4.3,-72.919998
2020-12-22 04:00:00,0.0,4.0,-4.0,-53.720001
...,...,...,...,...
2023-01-09 16:00:00,0.1,7.6,-7.5,1204.510010
2023-01-09 17:00:00,0.0,10.0,-10.0,1296.579956
2023-01-09 18:00:00,0.0,9.1,-9.1,1354.890015
2023-01-09 19:00:00,0.0,11.1,-11.1,1266.089966


In [13]:
N = 24*7
bat_capacity=13
Start="2022-06-19 00:00:00"
End = pd.date_range(start=Start, periods=N, freq="h")[-1]

train_prod = 2*7*24
train_cons = 3000

BeforeProd = pd.date_range(end=Start, periods=train_prod+1, freq="h")[0]
BeforeCons = pd.date_range(end=Start, periods=train_cons+1, freq="h")[0]

In [14]:
forecasterProd = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123,max_depth=8,max_features=3,n_estimators=100,n_jobs=4),
                 lags      = list(range(1,10+1,1))
                )
forecasterCons = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123,max_depth=8,max_features=4,n_estimators=100,n_jobs=4),
                 lags      = list(range(1,10+1,1))
                )

preds = pd.DataFrame()
all_preds = []
stepsize=24 
for i in range(N//stepsize):#stepsize has to be even
    _, predProd = backtesting_forecaster(forecaster = forecasterProd,
                                         y          = prod.loc[BeforeProd:End].iloc[i*stepsize:train_prod+stepsize*(i+1)]['prod_'+house],
                                         exog       = prod.loc[BeforeProd:End].iloc[i*stepsize:train_prod+stepsize*(i+1)].drop(['prod_'+house], axis=1),
                                         initial_train_size = train_prod,
                                         steps      = 24,
                                         refit      = True,
                                         metric     = 'mean_squared_error',
                                         fixed_train_size   = True,
                                         verbose    = False
                                        )

    _, predCons = backtesting_forecaster(forecaster = forecasterCons,
                                         y          = cons.loc[BeforeCons:End].iloc[i*stepsize:train_cons+stepsize*(i+1)]['cons_'+house],
                                         exog       = cons.loc[BeforeCons:End].iloc[i*stepsize:train_cons+stepsize*(i+1)].drop(['cons_'+house], axis=1),
                                         initial_train_size = train_cons,
                                         steps      = 24,
                                         refit      = True,
                                         metric     = 'mean_squared_error',
                                         fixed_train_size   = True,
                                         verbose    = False
                                        )
    predi = pd.merge(predProd, predCons, how="outer", left_index=True,right_index=True)
    predi =predi.rename(columns={"pred_x":"prod_"+house,"pred_y":"cons_"+house})

    predi["prod_"+house]=np.array((predi["prod_"+house].to_numpy())*10,dtype=int)/10
    predi["cons_"+house]=np.array((predi["cons_"+house].to_numpy())*10,dtype=int)/10

    predi["power_yield"] = predi["prod_"+house] - predi["cons_"+house]
    predi["SpotPriceDKK"] = merged.loc[predi.index[0]:predi.index[-1]]["SpotPriceDKK"]
    
    preds=pd.concat([preds,predi[:stepsize]])
preds

,prod_h16,cons_h16,power_yield,SpotPriceDKK
2022-06-19 00:00:00,0.0,6.3,-6.3,1540.390015
2022-06-19 01:00:00,0.0,5.7,-5.7,1413.410034
2022-06-19 02:00:00,0.0,4.7,-4.7,1231.300049
2022-06-19 03:00:00,0.0,4.0,-4.0,1039.969971
2022-06-19 04:00:00,0.0,3.7,-3.7,897.140015
...,...,...,...,...
2022-06-25 19:00:00,9.4,10.8,-1.4,2336.860107
2022-06-25 20:00:00,5.3,9.4,-4.1,2434.100098
2022-06-25 21:00:00,1.6,8.5,-6.9,2400.699951
2022-06-25 22:00:00,0.2,7.4,-7.2,2410.669922


In [17]:
battery_no = Battery(max_capacity=bat_capacity,max_charge=0)

series_no_battery = pred_logic_rollout(merged.loc[Start:End].iloc[0:N],preds.loc[Start:End].iloc[0:N],
                                       battery_no, logic_bat, get_price)

print_price_summary(series_no_battery)
logic_series_print(series_no_battery)

Cost for period: 2022-06-19 00:00:00 to 2022-06-25 23:00:00 is:  802.0  DKK
Average cost per year is: 42077.0 DKK
Number of kwh purchased in the period: 456.29999999990105
Average number of kwh purchased per year: 23951.651497000792
Average number of kwh sold per year: 34334.37485029512
hour     price    yield    surplus  buy      charge   before   after    cost     cumsum  
    0:   1.5404, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721
    1:   1.4134, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632
    2:   1.2313, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347
    3:   1.0400, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866
    4:   0.8971, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  3.4091, 28.4957
    5:   0.8254, -3.4000, -3.4000,  0.0000,  0.0000,  0.0000,  0.0000,  2.8065, 31.3022
    6:   0.8373, -3.5000, -3.5000,  0.0000,  0.0000,  0.0000,  0.0000,  2.9304, 34.2326
    7:

In [18]:
battery_true = Battery(max_capacity=bat_capacity,max_charge=7)

series_battery_true = pred_logic_rollout(merged.loc[Start:End].iloc[0:N],preds.loc[Start:End].iloc[0:N], 
                                         battery_true, logic_bat, get_price)

print_price_summary(series_battery_true)
logic_series_print(series_battery_true)

Cost for period: 2022-06-19 00:00:00 to 2022-06-25 23:00:00 is:  746.0  DKK
Average cost per year is: 39160.0 DKK
Number of kwh purchased in the period: 446.7999999997963
Average number of kwh purchased per year: 23452.98682633661
Average number of kwh sold per year: 31436.87065867286
hour     price    yield    surplus  buy      charge   before   after    cost     cumsum  
    0:   1.5404, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721
    1:   1.4134, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632
    2:   1.2313, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347
    3:   1.0400, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866
    4:   0.8971, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  3.4091, 28.4957
    5:   0.8254, -3.4000, -3.4000,  0.0000,  0.0000,  0.0000,  0.0000,  2.8065, 31.3022
    6:   0.8373, -3.5000, -3.5000,  0.0000,  0.0000,  0.0000,  0.0000,  2.9304, 34.2326
    7:  

  114:   2.7334,  8.5000,  7.9000,  0.0000,  0.6000, 12.3000, 12.9000, -2.1594,540.3035
  115:   3.1918,  2.7000,  6.0000,  0.0000, -3.3000, 12.2000,  8.8000, -1.9151,538.3884
  116:   3.0797, -0.6000,  3.7000,  0.0000, -4.3000,  8.3000,  4.0000, -1.1395,537.2489
  117:   2.6703, -4.7000, -0.9000,  0.0000, -3.8000,  3.8000,  0.0000,  2.4032,539.6521
  118:   2.1749, -6.5000, -6.5000,  0.0000,  0.0000,  0.0000,  0.0000, 14.1368,553.7890
  119:   1.5650, -7.6000, -7.6000,  0.0000,  0.0000,  0.0000,  0.0000, 11.8938,565.6827
  120:   1.6726, -5.6000, -5.6000,  0.0000,  0.0000,  0.0000,  0.0000,  9.3663,575.0490
  121:   1.5747, -5.4000, -5.4000,  0.0000,  0.0000,  0.0000,  0.0000,  8.5035,583.5526
  122:   1.5191, -4.5000, -4.5000,  0.0000,  0.0000,  0.0000,  0.0000,  6.8359,590.3885
  123:   1.4320, -3.7000, -3.7000,  0.0000,  0.0000,  0.0000,  0.0000,  5.2983,595.6868
  124:   1.4010, -3.4000, -3.4000,  0.0000,  0.0000,  0.0000,  0.0000,  4.7635,600.4503
  125:   1.4062, -3.5000, -3.500